### Test

In [42]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import unidecode
import re


In [175]:
codes = {'Atletico-Madrid':'db3b9613',
        'Mallorca':'2aa12281',
         'Hercules':'4222a514', 
         'Racing-Sant': 'dee3bbc8',
         'Malaga': '1c896955',
         'Deportivo-La-Coruna':'2a60ed82',
         'Levante': '9800b6a1',
         'Espanyol': 'a8661628',
         'Real-Sociedad':'e31d1cd9',
         'Osasuna': '03c57e2b',
         'Real-Madrid':'53a2f082',
         'Almeria':'78ecf4bb',
         'Villarreal':'2a8183b3',
         'Getafe':'7848bd64',
         'Sevilla':'ad2be733',
         'Zaragoza':'800303a0',
         'Valencia':'dcc91a7b',
         'Barcelona':'206d90db',
         'Athletic-Club':'2b390eca',
         'Sporting-Gijon':'bb9efd50',
         'Betis':'fc536746',
         'Rayo-Vallecano':'98e8af82',
         'Granada':'a0435291',
         'Celta-Vigo':'f25da7fb',
         'Valladolid':'17859612',
         'Elche':'6c8b07df',
         'Cordoba':'6009ff35',
         'Eibar':'bea5c710',
         'Las-Palmas':'0049d422',
         'Leganes':'7c6f2c78',
         'Alaves':'8d6fd021',
         'Girona':'9024a00a',
         'Huesca':'c6c493e6',
         'Cadiz':'ee7c297c',
         'Oviedo':'ab358912',
         'Numancia':'76d8bafa',
         'Tenerife':'27cc9c62',
         'Recreativo':'6e1d6d95',
         'Real-Murcia':'ab0dc306',
         'Albacete':'b7e3e46e',
         'Gimnastic':'d5de8ff3',
         'Xerez':'c653e8bf'
         
         
        }

In [140]:
teams = preprocess_names(teams)

### Working form for team previous matches

In [ ]:
URL = "https://fbref.com/en/squads/db3b9613/2006-2007/Atletico-Madrid-Stats"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

a = soup.find_all("table")[1]

new_table = pd.DataFrame(index=[0]) 
df= pd.DataFrame(index=[0]) 

row_marker = 0
for row in a.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
        le = len(column.get_text())
        addd = column.get_text()
        if le==0:
            addd = 0
        
        new_table[column.get('data-stat')] = str(addd)
    new_table['time'] = row.find_all('th')[0].get_text()
    df = pd.concat([new_table, df])

#### Get historical data

In [6]:
all_df = pd.DataFrame()
for i in range(len(years)):

    if i<len(years)-1:
        URL = "https://fbref.com/en/squads/db3b9613/"+str(years[i])+"-"+str(years[i+1])+"/Atletico-Madrid-Stats"
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, "html.parser")

        a = soup.find_all("table")[1]

        new_table = pd.DataFrame(index=[0]) 
        df= pd.DataFrame(index=[0]) 

        row_marker = 0
        for row in a.find_all('tr'):
            columns = row.find_all('td')
            for column in columns:
                le = len(column.get_text())
                addd = column.get_text()
                if le==0:
                    addd = 0

                new_table[column.get('data-stat')] = str(addd)
            new_table['time'] = row.find_all('th')[0].get_text()
            df = pd.concat([new_table, df])
        df['season'] = str(years[i])+"-"+str(years[i+1])
        all_df = pd.concat([all_df, df])

In [139]:
def preprocess_names(list_names):
    """
    Get a list of names in this case the fisrt letter 
    in capital and drop accents and fill spaces with '-'
    
    """
    drop_accents = np.vectorize(unidecode.unidecode)
    list_names = drop_accents(list_names)
    list_names = [i.replace(' ','-') if len(i.split(' '))>1 else i for i in list_names ]
    
    return list_names

In [188]:
def get_team_match_data(team_name, seasons):
    """
    
    Extract from page the previous mathces of the seasons input
    

    """
    not_found = []
    all_df = pd.DataFrame()
    for i in range(len(years)):

        if i<len(years)-1:
            URL = "https://fbref.com/en/squads/"+codes[team_name]+"/" \
            +str(years[i])+"-"+str(years[i+1])+"/"+team_name+"-Stats"
            
            print(URL)
            page = requests.get(URL)
            soup = BeautifulSoup(page.content, "html.parser")
            try:
                a = soup.find_all("table")[1]
            except:
                name = str(team_name)+'_'+str(years[1])+"-"+str(years[1+1])
                not_found.append(name)                
                continue
                
            new_table = pd.DataFrame(index=[0]) 
            df= pd.DataFrame(index=[0]) 

            row_marker = 0
            for row in a.find_all('tr'):
                columns = row.find_all('td')
                for column in columns:
                    le = len(column.get_text())
                    addd = column.get_text()
                    if le==0:
                        addd = 0

                    new_table[column.get('data-stat')] = str(addd)
                new_table['time'] = row.find_all('th')[0].get_text()
                df = pd.concat([new_table, df])
            df['season'] = str(years[i])+"-"+str(years[i+1])
            all_df = pd.concat([all_df, df])
    all_df.columns = all_df.columns.str.lower()
    all_df['team_name'] = team_name
    all_df.dropna(subset=['comp'], inplace=True)
    
    cols = ['team_name','time', 'comp', 'round', 'dayofweek'
            , 'venue', 'result', 'goals_for', 'goals_against'
            , 'opponent', 'possession','season']
    
    return all_df[cols], not_found

In [183]:
team_name = 'Granada'
years = np.linspace(2001, 2021, 21).astype(int)
a = get_team_match_data(team_name, years)

,team_name,time,comp,round,dayofweek,venue,result,goals_for,goals_against,opponent,possession,season
0,Granada,2012-05-13,La Liga,Matchweek 38,Sun,Away,L,0,1,Rayo Vallecano,0,2011-2012
0,Granada,2012-05-05,La Liga,Matchweek 37,Sat,Home,L,1,2,Real Madrid,0,2011-2012
0,Granada,2012-05-01,La Liga,Matchweek 20,Tue,Home,W,2,1,Espanyol,0,2011-2012
0,Granada,2012-04-28,La Liga,Matchweek 36,Sat,Away,L,1,3,Levante,0,2011-2012
0,Granada,2012-04-22,La Liga,Matchweek 35,Sun,Home,W,1,0,Getafe,0,2011-2012
...,...,...,...,...,...,...,...,...,...,...,...,...
0,Granada,2020-09-27,La Liga,Matchweek 3,Sun,Away,L,1,6,Atlético Madrid,45,2020-2021
0,Granada,2020-09-24,Europa Lg,Third qualifying round,Thu,Home,W,2,0,ge Lokomotivi Tbilisi,0,2020-2021
0,Granada,2020-09-20,La Liga,Matchweek 2,Sun,Home,W,2,1,Alavés,55,2020-2021
0,Granada,2020-09-17,Europa Lg,Second qualifying round,Thu,Away,W,4,0,al Teuta Durrës,0,2020-2021


In [ ]:
years = np.linspace(2001, 2021, 21).astype(int)
team_data = pd.DataFrame()
not_in_df = []
for i in codes.keys():
    team_data_tmp, not_found = get_team_match_data(i, years)
    team_data = pd.concat([team_data_tmp, team_data])
    not_in_df.append(not_found)

In [197]:
team_data.to_pickle('matches_laliga_2001-2021.pikcle')

In [3]:
team_data = pd.read_pickle('matches_laliga_2001-2021.pikcle')

In [4]:
team_data

,team_name,time,comp,round,dayofweek,venue,result,goals_for,goals_against,opponent,possession,season
0,Xerez,2010-05-16,La Liga,Matchweek 38,Sun,Away,D,1,1,Osasuna,0,2009-2010
0,Xerez,2010-05-08,La Liga,Matchweek 37,Sat,Home,W,3,2,Zaragoza,0,2009-2010
0,Xerez,2010-05-04,La Liga,Matchweek 36,Tue,Away,L,1,3,Valencia,0,2009-2010
0,Xerez,2010-05-01,La Liga,Matchweek 35,Sat,Home,W,2,1,Almería,0,2009-2010
0,Xerez,2010-04-24,La Liga,Matchweek 34,Sat,Away,L,1,3,Barcelona,0,2009-2010
...,...,...,...,...,...,...,...,...,...,...,...,...
0,Atletico-Madrid,2020-10-21,Champions Lg,Group stage,Wed,Away,L,0,4,de Bayern Munich,42,2020-2021
0,Atletico-Madrid,2020-10-17,La Liga,Matchweek 6,Sat,Away,W,2,0,Celta Vigo,50,2020-2021
0,Atletico-Madrid,2020-10-03,La Liga,Matchweek 5,Sat,Home,D,0,0,Villarreal,54,2020-2021
0,Atletico-Madrid,2020-09-30,La Liga,Matchweek 4,Wed,Away,D,0,0,Huesca,63,2020-2021


### Feature repository
- Resultados últimos cinco partidos
- Resultados partidos previos por liga de ese equipo
- Goles a favor y en contra en liga 
- Goles anotados a ese equipo, diferencia de goles (media, media ponderada, último partido)
- Días desde último partido
- Posiciones, historico de posiciones en las temporadas pasadas
- Sacar en que posición de la liga va en ese partido en especifico cada equipo para ponerlo como variable.
- Categorica si va en posición de descenso o posición de champions
- (Optional) A cuantos puntos está de descenso (3 últimos) y de posición de ligas europeas (Asumir top 5)
- Está jugando champions?


### Additional data
- Buscar inversión para esa temporada
- Cambio de entrenador 

### Variable construction

#### Results of previous n matches

Logic take any column with the name of the team and build with that date 

In [7]:
la_liga_df = team_data[team_data['comp']=='La Liga']

ej:  Xerez  vs Osasuna -> 2010-05-16

In [110]:
def match_win_loss(name:str, date_match:str, n_previous_matches:int, la_liga_df, local_away:str):
    
    la_liga_df.sort_values('time', ascending=False, inplace=True)

    matches = la_liga_df[(la_liga_df['team_name']==name)
                           &(la_liga_df['time']<date_match)].head(n_previous_matches)
    
    matches['new_result'] = np.where(matches['result']=='W', 'win'
         , np.where(matches['result']=='L', 'loss', np.where(matches['result']=='D', 'draw', 'None')))

    ### Previous matches win, loss and draw by venue (home, away)
    win_loss = matches.groupby(['new_result','venue']).count().T.iloc[[0]]
    win_loss.columns = [str(i[0])+'_'+str(i[1]) for i in win_loss.columns]
    win_loss.columns = win_loss.columns.str.lower()
    win_loss.reset_index(drop=True, inplace=True)

    ### Previous matches win, loss total
    win_loss_all = matches.groupby(['new_result']).count().T.iloc[[0]]
    win_loss_all.reset_index(drop=True, inplace=True)
    win_loss = pd.concat([win_loss_all,win_loss], axis=1)
    ### percentages
    win_loss['percentage_win'] = win_loss['win']/n_previous_matches
    win_loss['percentage_loss'] = win_loss['loss']/n_previous_matches
    win_loss['percentage_draw'] = win_loss['draw']/n_previous_matches
    print(win_loss)
    win_loss['percentage_home_win'] = win_loss['win_home']/n_previous_matches
    win_loss['percentage_home_loss'] = win_loss['loss_home']/n_previous_matches
    win_loss['percentage_home_draw'] = win_loss['draw_home']/n_previous_matches

    win_loss['percentage_away_win'] = win_loss['win_away']/n_previous_matches
    win_loss['percentage_away_loss'] = win_loss['loss_away']/n_previous_matches
    win_loss['percentage_away_draw'] = win_loss['draw_away']/n_previous_matches
    
    win_loss.columns = [i+'_'+local_away for i in win_loss.columns]
    return win_loss

In [111]:
name = 'Osasuna'
date_match = '2010-05-16'
n_previous_matches = 10
local_away = 'away'

In [112]:
df = match_win_loss(name, date_match, n_previous_matches, la_liga_df, local_away)

   draw  loss  win  draw_home  loss_away  win_home  percentage_win  \
0     2     5    3          2          5         3             0.3   

   percentage_loss  percentage_draw  
0              0.5              0.2  


/home/oleon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


KeyError: 'loss_home'

In [106]:
df

,draw_local,loss_local,win_local,draw_away_local,draw_home_local,loss_away_local,loss_home_local,win_away_local,win_home_local,percentage_win_local,percentage_loss_local,percentage_draw_local,percentage_home_win_local,percentage_home_loss_local,percentage_home_draw_local,percentage_away_win_local,percentage_away_loss_local,percentage_away_draw_local
0,3,3,4,2,1,2,1,1,3,0.4,0.3,0.3,0.3,0.1,0.1,0.1,0.2,0.2


In [113]:
la_liga_df.sort_values('time', ascending=False, inplace=True)

matches = la_liga_df[(la_liga_df['team_name']==name)
                           &(la_liga_df['time']<date_match)].head(n_previous_matches)


/home/oleon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [114]:
matches

,team_name,time,comp,round,dayofweek,venue,result,goals_for,goals_against,opponent,possession,season
0,Osasuna,2010-05-08,La Liga,Matchweek 37,Sat,Away,L,1,2,Espanyol,0,2009-2010
0,Osasuna,2010-05-05,La Liga,Matchweek 36,Wed,Home,W,3,1,La Coruña,0,2009-2010
0,Osasuna,2010-05-02,La Liga,Matchweek 35,Sun,Away,L,2,3,Real Madrid,0,2009-2010
0,Osasuna,2010-04-26,La Liga,Matchweek 34,Mon,Home,D,0,0,Athletic Club,0,2009-2010
0,Osasuna,2010-04-19,La Liga,Matchweek 33,Mon,Away,L,0,2,Mallorca,0,2009-2010
0,Osasuna,2010-04-14,La Liga,Matchweek 32,Wed,Home,D,2,2,Málaga,0,2009-2010
0,Osasuna,2010-04-11,La Liga,Matchweek 31,Sun,Home,W,2,0,Zaragoza,0,2009-2010
0,Osasuna,2010-04-04,La Liga,Matchweek 30,Sun,Away,L,0,3,Valencia,0,2009-2010
0,Osasuna,2010-03-28,La Liga,Matchweek 29,Sun,Home,W,1,0,Almería,0,2009-2010
0,Osasuna,2010-03-24,La Liga,Matchweek 28,Wed,Away,L,0,2,Barcelona,0,2009-2010


In [117]:
matches['new_result'] = np.where(matches['result']=='W', 'win'
         , np.where(matches['result']=='L', 'loss', np.where(matches['result']=='D', 'draw', 'None')))

### Previous matches win, loss and draw by venue (home, away)
win_loss = matches.groupby(['new_result','venue']).count().T.iloc[[0]]


In [118]:
win_loss

new_result,draw,loss,win
venue,Home,Away,Home
team_name,2,5,3


In [116]:
win_loss.columns = [str(i[0])+'_'+str(i[1]) for i in win_loss.columns]
win_loss.columns = win_loss.columns.str.lower()
win_loss.reset_index(drop=True, inplace=True)

### Previous matches win, loss total
win_loss_all = matches.groupby(['new_result']).count().T.iloc[[0]]
win_loss_all.reset_index(drop=True, inplace=True)
win_loss = pd.concat([win_loss_all,win_loss], axis=1)

,draw,loss,win,draw_home,loss_away,win_home
0,2,5,3,2,5,3


In [97]:
win_loss['percentage_win'] = win_loss['win']/n_previous_matches
win_loss['percentage_loss'] = win_loss['loss']/n_previous_matches
win_loss['percentage_draw'] = win_loss['draw']/n_previous_matches

win_loss['percentage_home_win'] = win_loss['win_home']/n_previous_matches
win_loss['percentage_home_loss'] = win_loss['loss_home']/n_previous_matches
win_loss['percentage_home_draw'] = win_loss['draw_home']/n_previous_matches

win_loss['percentage_away_win'] = win_loss['win_away']/n_previous_matches
win_loss['percentage_away_loss'] = win_loss['loss_away']/n_previous_matches
win_loss['percentage_away_draw'] = win_loss['draw_away']/n_previous_matches

In [100]:
[i+'_'+'local' for i in win_loss.columns]

['draw_local',
 'loss_local',
 'win_local',
 'draw_away_local',
 'draw_home_local',
 'loss_away_local',
 'loss_home_local',
 'win_away_local',
 'win_home_local',
 'percentage_win_local',
 'percentage_loss_local',
 'percentage_draw_local',
 'percentage_home_win_local',
 'percentage_home_loss_local',
 'percentage_home_draw_local',
 'percentage_away_win_local',
 'percentage_away_loss_local',
 'percentage_away_draw_local']

## Transfer information


In [152]:
URL = "https://www.transfermarkt.com/fc-barcelona/alletransfers/verein/131"
headers = {"User-Agent":"Mozilla/5.0"}
page = requests.get(URL, headers= headers)
soup = BeautifulSoup(page.content, "html.parser")

a = soup.find_all(class_ = "box")
tables = soup.find_all("table")


In [8]:
tables_header = soup.find_all(class_ = "table-header")


In [12]:
company = soup.select('h2')[6].text.strip()
print(company)

Arrivals 19/20


In [192]:
soup.find_all(class_ = 'box')[1].select('h2')

[<h2>
                         Arrivals 22/23                    </h2>]

In [161]:
new_table = pd.DataFrame(index=[0]) 
df= pd.DataFrame(index=[0]) 
col_names=['name', 'drop', 'from_to', 'value']

row_marker = 0
all_df = pd.DataFrame(index=[0]) 
soup.find_all(class_ = 'box')[1].select('h2')
for row in tables[0].find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    if len(columns)==1:
        continue
    for column in columns:
        le = len(column.get_text())
        addd = column.get_text()
        if le==0:
            addd = 0
        new_table[col_names[column_marker]] = str(addd)
        column_marker =column_marker + 1
    df = pd.concat([new_table, df])
df.dropna(inplace=True)
df.drop(columns=['drop'], inplace=True)
## get season and arrival or departure
df['type'] = soup.select('h2')[4].text.strip().split(' ')[0]
df['season'] = soup.select('h2')[4].text.strip().split(' ')[1]
## get value
a = [(re.findall(r'[0-9]+', df['value'].iloc[i])) for i in range(len(df))]
df['value_money'] = ['.'.join(a[i]) for i in range(len(a))]

df['value_money'] = np.where(df['value_money']=='', 0, df['value_money'])
df['value_money'] = df['value_money'].astype(float)

df['multiplier'] = np.where(df['value'].str.endswith('m')
                            , 1e6
                            , np.where(df['value'].str.endswith('Th.'), 1e3,0))

df['value_money'] = df['multiplier']*df['value_money']
df['number_of_loans'] = df['value'].str.lower().str.contains('loan').sum()
df['number_of_free_transfers'] = df['value'].str.lower().str.contains('free').sum()
df['total_amount'] = df['value_money'].sum()
all_df = pd.concat([df, all_df])

## Get all info from tables departures arrivals

In [282]:
new_table = pd.DataFrame(index=[0]) 
col_names=['name', 'drop', 'from_to', 'value']

row_marker = 0
all_df = pd.DataFrame(index=[0]) 
for k in range(len(soup.find_all(class_ = 'box'))):
    df= pd.DataFrame(index=[0]) 
    table = soup.find_all(class_ = 'box')[k].select('table')
    if soup.find_all(class_ = 'box')[k].select('h2') and table:
        type_table = soup.find_all(class_ = 'box')[k].select('h2')[0].text.strip('\n').strip(' ').split(' ')[0]
        season_table = soup.find_all(class_ = 'box')[k].select('h2')[0].text.strip('\n').strip(' ').split(' ')[1]


        for row in table[0].find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            if len(columns)==1:
                continue
            for column in columns:
                le = len(column.get_text())
                addd = column.get_text()
                if le==0:
                    addd = 0
                new_table[col_names[column_marker]] = str(addd)
                column_marker =column_marker + 1
            df = pd.concat([new_table, df])
        df.dropna(inplace=True)
        df.drop(columns=['drop'], inplace=True)
        ## get season and arrival or departure
        df['type'] = type_table
        df['season'] = season_table
        ## get value
        a = [(re.findall(r'[0-9]+', df['value'].iloc[i])) for i in range(len(df))]
        df['value_money'] = ['.'.join(a[i]) for i in range(len(a))]

        df['value_money'] = np.where(df['value_money']=='', 0, df['value_money'])
        df['value_money'] = df['value_money'].astype(float)

        df['multiplier'] = np.where(df['value'].str.endswith('m')
                                    , 1e6
                                    , np.where(df['value'].str.endswith('Th.'), 1e3,0))

        df['value_money'] = df['multiplier']*df['value_money']
        df['number_of_loans'] = df['value'].str.lower().str.contains('loan').sum()
        df['number_of_free_transfers'] = df['value'].str.lower().str.contains('free').sum()
        df['total_amount'] = df['value_money'].sum()
        all_df = pd.concat([df, all_df])
    else:
        continue

In [287]:
URL = "https://www.transfermarkt.com/fc-barcelona/alletransfers/verein/131"
https://www.transfermarkt.com/fc-sevilla/alletransfers/verein/368
    https://www.transfermarkt.com/real-betis/alletransfers/verein/150/
codes_transfermrkt = {
    
    'fc-barcelona':'131',
    'fc-sevilla':'368',
    'real-betis':'150'
    
    
    
    
}

227

In [284]:
type_table

'Arrivals'

In [285]:
all_df[all_df['season'] =='21/22']

,name,from_to,value,type,season,value_money,multiplier,number_of_loans,number_of_free_transfers,total_amount
0,Francisco Trincão,Wolves,loan transfer,Departures,21/22,0.0,0.0,3.0,4.0,63500000.0
0,Miralem Pjanic,Besiktas,loan transfer,Departures,21/22,0.0,0.0,3.0,4.0,63500000.0
0,Lionel Messi,Paris SG,free transfer,Departures,21/22,0.0,0.0,3.0,4.0,63500000.0
0,Monchu,Granada CF,free transfer,Departures,21/22,0.0,0.0,3.0,4.0,63500000.0
0,Matheus Fernandes,Palmeiras,free transfer,Departures,21/22,0.0,0.0,3.0,4.0,63500000.0
0,Juan Miranda,Real Betis,free transfer,Departures,21/22,0.0,0.0,3.0,4.0,63500000.0
0,Carles Aleñá,Getafe,€5.00m,Departures,21/22,5000000.0,1000000.0,3.0,4.0,63500000.0
0,Jean-Clair Todibo,OGC Nice,€8.50m,Departures,21/22,8500000.0,1000000.0,3.0,4.0,63500000.0
0,Antoine Griezmann,Atlético Madrid,Loan fee:€10.00m,Departures,21/22,10000000.0,1000000.0,3.0,4.0,63500000.0
0,Junior Firpo,Leeds,€15.00m,Departures,21/22,15000000.0,1000000.0,3.0,4.0,63500000.0


In [289]:


URL = 'https://www.transfermarkt.com/laliga/startseite/wettbewerb/ES1'
headers = {"User-Agent":"Mozilla/5.0"}
page = requests.get(URL, headers= headers)
soup = BeautifulSoup(page.content, "html.parser")


In [293]:
soup

<!DOCTYPE html>

<html class="no-js" lang="en">
<head>
<script type="text/javascript">
    !function () { var e = function () { var e, t = "__tcfapiLocator", a = [], n = window; for (; n;) { try { if (n.frames[t]) { e = n; break } } catch (e) { } if (n === window.top) break; n = n.parent } e || (!function e() { var a = n.document, r = !!n.frames[t]; if (!r) if (a.body) { var i = a.createElement("iframe"); i.style.cssText = "display:none", i.name = t, a.body.appendChild(i) } else setTimeout(e, 5); return !r }(), n.__tcfapi = function () { for (var e, t = arguments.length, n = new Array(t), r = 0; r < t; r++)n[r] = arguments[r]; if (!n.length) return a; if ("setGdprApplies" === n[0]) n.length > 3 && 2 === parseInt(n[1], 10) && "boolean" == typeof n[3] && (e = n[3], "function" == typeof n[2] && n[2]("set", !0)); else if ("ping" === n[0]) { var i = { gdprApplies: e, cmpLoaded: !1, cmpStatus: "stub" }; "function" == typeof n[2] && n[2](i) } else a.push(n) }, n.addEventListener("message", (f

In [294]:
soup.select('link')

[<link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>,
 <link href="/favicon-16x16.png" rel="shortcut icon" sizes="16x16"/>,
 <link href="/android-chrome-192x192.png" rel="shortcut icon" sizes="192x192"/>,
 <link href="/apple-touch-icon-152x152.png" rel="apple-touch-icon-precomposed"/>,
 <link href="https://www.transfermarkt.de/primera-division/startseite/wettbewerb/ES1" hreflang="de" rel="alternate">
 <link href="https://www.transfermarkt.de/primera-division/startseite/wettbewerb/ES1" hreflang="de-LU" rel="alternate">
 <link href="https://www.transfermarkt.at/primera-division/startseite/wettbewerb/ES1" hreflang="de-AT" rel="alternate">
 <link href="https://www.transfermarkt.ch/primera-division/startseite/wettbewerb/ES1" hreflang="de-CH" rel="alternate"/>
 <link href="https://www.transfermarkt.com.tr/primera-division/startseite/wettbewerb/ES1" hreflang="tr" rel="alternate"/>
 <link href="https://www.transfermarkt.it/primera-division/startseite/wettbewerb/ES1" hreflang="i

In [303]:
soup.find_all(class_ = 'box')[2].select('table')

[<table class="items">
 <thead>
 <tr>
 <th colspan="2" id="yw1_c0">club</th><th class="hide" id="yw1_c1"><a class="sort-link" href="/primera-division/startseite/wettbewerb/ES1/sort/name">name</a></th><th class="hide" id="yw1_c2"><a class="sort-link" href="/primera-division/startseite/wettbewerb/ES1/sort/name">name</a></th><th class="zentriert" id="yw1_c3"><a class="sort-link" href="/primera-division/startseite/wettbewerb/ES1/sort/anzahl_spieler_hidden.desc">Squad</a></th><th class="zentriert hide-for-small hide-for-pad" id="yw1_c4"><a class="sort-link" href="/primera-division/startseite/wettbewerb/ES1/sort/alter_durchschnitt.desc">ø age</a></th><th class="zentriert hide-for-pad hide-for-small" id="yw1_c5"><a class="sort-link" href="/primera-division/startseite/wettbewerb/ES1/sort/legionaere.desc">Foreigners</a></th><th class="rechts hide-for-small hide-for-pad" id="yw1_c6"><a class="sort-link" href="/primera-division/startseite/wettbewerb/ES1/sort/marktwert_schnitt_anzeige.desc">ø mark